Question 2
* Consider a continuous attribute in your dataset as the target variable, perform
 regression analysis using different ensemble methods, visualize and interpret the results. 


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, StackingClassifier, BaggingRegressor, AdaBoostRegressor, StackingRegressor, RandomForestRegressor,GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from statistics import variance


In [4]:
df = pd.read_csv("heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Determine the age of the patent using the above data

In [5]:
# Data Prep
X = df.drop(["age"], axis=1)
y = df["age"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [7]:
# Models to run

models = {
    "LR" : LinearRegression(),
    "SVR_Linear" : SVR(kernel="linear"),
    "SVR_rbf" : SVR(kernel="rbf"),
    "DTR" : DecisionTreeRegressor(),
    "RFR" : RandomForestRegressor(),
    "GBR" : GradientBoostingRegressor()
}

In [8]:
rmse_values_bag = {}
for key in models:
    model = BaggingRegressor(base_estimator=models[key], n_estimators=50, max_samples=0.8, max_features=0.8)
    model.fit(X_train, y_train)
    predicted_values = model.predict(X_test)
    mse = mean_squared_error(predicted_values, y_test)
    rmse = np.sqrt(mse)
    rmse_values_bag[key] = rmse

In [9]:
pd.DataFrame([rmse_values_bag])

,LR,SVR_Linear,SVR_rbf,DTR,RFR,GBR
0,9.085851,9.208513,9.737562,8.887033,8.759469,9.037808


In [10]:
rmse_values_boo = {}
for key in models:
    model = AdaBoostRegressor(base_estimator=models[key], n_estimators=50)
    model.fit(X_train, y_train)
    predicted_values = model.predict(X_test)
    mse = mean_squared_error(predicted_values, y_test)
    rmse = np.sqrt(mse)
    rmse_values_boo[key] = rmse

In [11]:
pd.DataFrame([rmse_values_boo])

,LR,SVR_Linear,SVR_rbf,DTR,RFR,GBR
0,9.113361,8.870254,9.382275,9.334407,8.926996,9.086082


In [12]:
base_learners = [
                 ('cf_1', LinearRegression()),
                 ('cf_2', DecisionTreeRegressor()),             
                 ('cf_3', GradientBoostingRegressor()),             
                 ('cf_4', SVR(kernel="linear")),             
                 ('cf_5', SVR(kernel="rbf"))           
                ]
final_estimator = RandomForestRegressor(n_estimators=10, random_state=42)

In [13]:
model = StackingRegressor(estimators=base_learners, final_estimator=final_estimator)
model.fit(X_train, y_train)
predicted_values = model.predict(X_test)
mse = mean_squared_error(predicted_values, y_test)
rmse = np.sqrt(mse)

In [14]:
pd.DataFrame([rmse])

,0
0,8.672635


In [19]:
x_bar = np.mean(y)
vari = variance(y)
std_dev = np.std(y)
Low = x_bar-2*std_dev
Up = x_bar+2*std_dev
outliers_l = []
for j in y:
    if j > Up or j < Low:
        outliers_l.append(j)
outlier_count = len(outliers_l)
output_count  = len(y)
percentage_outliers = ((len(outliers_l)/len(y))*100)
pd.DataFrame([[outlier_count, output_count, percentage_outliers]], columns=["Outliers Count", "Total Count", "Percentange"])

,Outliers Count,Total Count,Percentange
0,10,303,3.30033


The Percentage of out liers are very less , hence we can ignore the outliers

As `RandomForestRegressior` is best performing in bagging, bosting and stacking, we can use the random forest regressor for predictions, but as this data if more for classifcation problems its difficult to get accurate predictions over it.